In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["XLA_FLAGS"] = "--xla_gpu_cuda_data_dir='/usr/lib/cuda'"

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
!echo $CUDA_HOME $LD_LIBRARY_PATH $PATH

In [ ]:
!export CUDA_HOME=/usr/local/cuda
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CUDA_HOME/lib64
!export PATH=$PATH:$CUDA_HOME/bin

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import keras
from keras import models
import pandas as pd
import numpy as np
import sklearn
import keras.backend as K

IMAGE BIND

In [ ]:
cd  /home/ankr/Notebook/HateMM/ImageBind/

In [ ]:
from imagebind import data
import torch
from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
df = pd.read_excel('/home/ankr/Notebook/HateMM/Own_Explicit_Hate_Videos.xlsx')
df.head()

In [ ]:
eht_df = pd.read_csv('/home/ankr/Notebook/transcriptions_OWN_EXPLICIT_HATE_VIDEOS.csv')

eht_df.head()

In [ ]:
# Instantiate model
model = imagebind_model.imagebind_huge(pretrained=True)
model.eval()
model.to(device)

In [ ]:
class ModelExporter(torch.nn.Module):
    def __init__(self, model, modality):
        super().__init__()
        self.model = model
        self.modality = modality

    def forward(self, data):
        return self.model({self.modality: data})

In [ ]:
import openvino as ov

core = ov.Core()

In [ ]:
import ipywidgets as widgets

device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)

device

In [ ]:
from pathlib import Path

In [ ]:
FOLDER_NAME = '/home/ankr/Notebook/'

count = 0

# Iterate over filenames in df and retrieve transcriptions
for filename in df['Video_ID'][137:138]:
    print(f"Processing {filename}")
    text_list = []
    file = filename + '.wav'
    transcription = eht_df.loc[eht_df['file_name'] == file, 'Transcription'].values
    if len(transcription) > 0:
        for j in range(explicit_hate_frames[count]):
            text_list.append(transcription)
    else:
        print(f"No transcription found for {file}")

    print("Text completed")
    
    #audio_paths must be paths to audio files
    audio_paths = []
    file = filename + '.wav'
    for j in range(explicit_hate_frames[count]):
        audio_paths.append('/home/ankr/Notebook/HateMM/Own Videos Audio Files/Explicit Hate/' + file)

    print("Audio completed")
    
    #image_paths should be paths to images
    image_paths = []
    for i in range(len(os.listdir('/home/ankr/Notebook/HateMM/Own Image Frames/Explicit Hate Frames/' + filename))):
        image_paths.append('/home/ankr/Notebook/HateMM/Own Image Frames/Explicit Hate Frames/' + filename + '/' + f'frame_{i}'+ '.jpg')

    print("Image completed")

    # Convert all elements in text_list to strings
    text_list = [str(text) for text in text_list]

    inputs = {
        ModalityType.TEXT: data.load_and_transform_text(text_list, 'cpu'),
        ModalityType.AUDIO: data.load_and_transform_audio_data(audio_paths, 'cpu'),
        ModalityType.VISION: data.load_and_transform_vision_data(image_paths, 'cpu'),
    }

    with torch.no_grad():
        embeddings = model(inputs)
        
    print("Data loading done")
    ov_modality_models = {}

    modalities = [ModalityType.TEXT, ModalityType.VISION, ModalityType.AUDIO]
    for modality in modalities:
        export_dir = Path(f"image-bind-{modality}")
        file_name = f"image-bind-{modality}"
        export_dir.mkdir(exist_ok=True)
        ir_path = export_dir / f"{file_name}.xml"
        if not ir_path.exists():
            exportable_model = ModelExporter(model, modality)
            model_input = inputs[modality]
            ov_model = ov.convert_model(exportable_model, example_input=model_input)
            ov.save_model(ov_model, ir_path)
        else:
            ov_model = core.read_model(ir_path)
        ov_modality_models[modality] = core.compile_model(ov_model, device.value)
        
    embeddings = {}
    for modality in modalities:
        embeddings[modality] = ov_modality_models[modality](inputs[modality])[ov_modality_models[modality].output(0)]
        
    count = count+1
    print("Completed: " , count)
    file_to_store = f"/home/ankr/Notebook/HateMM/Own Videos ImageBind Embeddings/Explicit Hate/OWN_imagebind_EXPLICIT_HATE_embeddings_{count}.npy"
    
    # Save the embedding to file
    np.save(file_to_store, embeddings)
        
    # Getting shape and size of each array
    text_shape = embeddings['text'].shape
    text_size = embeddings['text'].size

    vision_shape = embeddings['vision'].shape
    vision_size = embeddings['vision'].size

    audio_shape = embeddings['audio'].shape
    audio_size = embeddings['audio'].size

    print("Text Shape:", text_shape)
    print("Text Size:", text_size)
 
    print("Vision Shape:", vision_shape)
    print("Vision Size:", vision_size)

    print("Audio Shape:", audio_shape)
    print("Audio Size:", audio_size)